In [14]:
#Starting imports
import pandas as pd 
import numpy as np 
import datetime as dt 
import math 
import warnings

warnings.filterwarnings("ignore") #Simply used to ignore all warnings

In [3]:
#Pulling data from Yahoo
import pandas_datareader as web

stocks = []
f = open("symbols.txt")
for line in f:
    stocks.append(line.strip())

web.DataReader(stocks, "yahoo", start="2010-1-1", end="2019-12-31")["Adj Close"].to_csv("prices.csv")
web.DataReader(stocks, "yahoo", start="2010-1-1", end = "2019-12-31")["Volume"].to_csv('volume.csv')

In [18]:
#Creating Global variables

prices = pd.read_csv("prices.csv", index_col="Date", parse_dates=True)
#.pct_change() calculates the percent change from previous row, multiply by 100 to get a %
volumesChange = pd.read_csv("volume.csv", index_col="Date", parse_dates=True).pct_change()*100
today = dt.date(2010, 1, 1)
end_day = dt.date(2019, 12, 31)

tickers = []
transaction_id = 0
money = 1000000 #Starting value, set to whatever you like

portfolio = {}
active_trades = []
transaction_log = []


In [31]:
#Main functions, get price, create transaction, buy stock, sell stock

#get stock price on a given date
def get_price(date, ticker):
    global prices
    return prices.loc[date][ticker]

#Transaction function to track buys and sells in a dict, as well as id, ticker, amount, price, date, etc.
def transaction(id, ticker, type, price, amount, info):
    global transaction_id
    if type == 'buy':
        exp_date = today + dt.timedelta(days=14)#How long until reevaluating whether to buy/sell
    else:
        exp_date = day
    
    if type == 'sell':
        data = {"id":id, "ticker": ticker, "type": type, "price":price, "amount":amount, "date":today, "exp_date":exp_date, "info":info}
        transaction_log.append(data)
    elif type == 'buy':
        data = {"id":transaction_id, "ticker": ticker, "type": type, "price":price, "amount":amount, "data":today, "exp_date":exp_date, "info":info}
        active_trades.append(data)
        transaction_log.append(data)
    


#Purchase function. buyList will be a list of ticker symbols, funds is amount being set aside to purchase stocks
def buy(buyList, funds):
    global money, portfolio
    for i in buyList:
        price = get_price(today, i)
        if not np.isnan(price):
            quantity = math.floor(funds/price)
            money -= quantity * price
            if i in portfolio:
                portfolio[i] += quantity
            else:
                portfolio[i] = quantity
            transaction(0, i, quantity, price, "buy", "") #logs to transaction_log


#Sell function (will be similar to buy)
def sell():
    global money, portfolio, prices, today
    to_remove = []
    for i in range(len(active_trades)):
        trade = active_trades[i]
        
        if trade["exp_date"] <= today and trade["type"] == buy:
            current_price = get_price(start_day, trade["ticker"])
            if not np.isnan(current_price):
                money += trade["amount"]*current_price
                portfolio[trade["ticker"]] -= trade["amount"]
                transaction( trade[id], trade[ticker], trade["amount"], current_price, "sell", trade["info"] )
                to_remove.append(i)
            else:
                trade["exp_date"] += dt.timedelta(days=1) #if not possible to sell today, will try again on the next day.

    #removes trades from active_trades once they have been sold
    to_remove.reverse()
    for j in to_remove:
        active_trades.remove(active_trades[j])


In [20]:
#Testing a strategy simulation
def sim():
    global money, volumesChange, today
    start_date = today - dt.timedelta(days=14)
    series = volumesChange.loc[start_date:today].mean()
    buyList = series[series>100].index.tolist()
    sell()
    if len(buyList) > 0:
        allocated_funds = 500000 / len(buyList) #Equal amount of funds per stock in buyList
        buy(buyList, allocated_funds)


In [27]:
#Helper functions: get_indices, is_tradeable valuation

#store all ticker symbols from symbols.txt in an array
def get_indices():
    global tickers
    f = open("symbols.txt")
    for line in f:
        tickers.append(line.strip())
    f.close()

#checks if date is a trading day, i.e. markets are open
def is_tradeable():
    global prices, today
    return np.datetime64(today) in list(prices.index.values)

#returns total value of cash + assets 
def valuation():
    global money, portfolio, today, prices
    total = money
    for t in tickers:
        current_price = get_price(today, t)
        if not np.isnan(current_price):
            total += portfolio[t]*current_price
    return int(total*100)/100 #rounds to nearest hundredth


In [28]:
#Main function
def main():
    global today
    get_indices
    for t in tickers:
        portfolio[t] = 0
    while today < end_day:
        while not is_tradeable():
            today += dt.timedelta(days=1) #Waits 1 day before attempting to trade again if not a tradeable day
        sim()
        current_value = valuation()
        print(current_value, today)
        today += dt.timedelta(days=7)


In [32]:
main()

-999950.0 2010-01-11
-1499950.0 2010-01-19
-1999938.73 2010-01-26
-2499907.12 2010-02-02
-2999895.85 2010-02-09
-3499884.73 2010-02-16
-3999884.24 2010-02-23
-4499831.69 2010-03-02
-4999751.62 2010-03-09
-5499746.66 2010-03-16
-5999704.38 2010-03-23
-6499624.13 2010-03-30
-6999577.4 2010-04-06
-7499534.92 2010-04-13
-7999473.85 2010-04-20
-8499331.91 2010-04-27
-8999301.85 2010-05-04
-9499251.52 2010-05-11
-9999228.17 2010-05-18
-10499196.17 2010-05-25
-10999183.98 2010-06-01
-11499161.59 2010-06-08
-11999135.59 2010-06-15
-12499117.59 2010-06-22
-12999099.59 2010-06-29
-13499071.98 2010-07-06
-13999042.15 2010-07-13
-14498955.43 2010-07-20
-14998937.43 2010-07-27
-15498933.68 2010-08-03
-15998926.19 2010-08-10
-16498893.14 2010-08-17
-16998882.54 2010-08-24
-17498858.31 2010-08-31
-17998828.31 2010-09-07
-18498810.31 2010-09-14
-18998796.31 2010-09-21
-19498781.16 2010-09-28
-19998704.42 2010-10-05
-20498670.6 2010-10-12
-20998618.8 2010-10-19
-20998618.8 2010-10-26
-21498611.45 2010-